In [1]:
import torch
print(torch.__version__)

2.7.1+cu118


In [3]:
! mkdir -p ~/work/transformer_chatbot/data/ && cd ~/work/transformer_chatbot/data/
! wget https://github.com/songys/Chatbot_data/raw/master/ChatbotData.csv

--2025-11-04 07:24:31--  https://github.com/songys/Chatbot_data/raw/master/ChatbotData.csv
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv [following]
--2025-11-04 07:24:32--  https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 889842 (869K) [text/plain]
Saving to: ‘ChatbotData.csv’

ChatbotData.csv     100%[===================>] 868.99K  5.29MB/s    in 0.2s    

2025-11-04 07:24:32 (5.29 MB/s) - ‘ChatbotData.csv’ saved [889842/889842]



In [4]:
# 폴더/패키지 준비
!pip -q install transformers==4.43.3 accelerate sentencepiece datasets
import os, pandas as pd, torch, numpy as np
print(torch.__version__, torch.cuda.is_available())


2.7.1+cu118 True


In [6]:
# 데이터 확인 및 정리

import pandas as pd, re
path = os.path.expanduser('~/work/transformer_chatbot/data/ChatbotData.csv')
df = pd.read_csv(path)
df = df.rename(columns={df.columns[0]:'Q', df.columns[1]:'A'})  # 안전하게
df = df[['Q','A']].dropna().drop_duplicates()

# 간단 클린업(이모지/이상문자 제거·연속 공백 축소)
def clean(s):
    s = re.sub(r'[^\S\r\n]+', ' ', str(s)).strip()
    s = re.sub(r'[\u200b-\u200f]', '', s)
    return s
df['Q'] = df['Q'].map(clean)
df['A'] = df['A'].map(clean)

# 길이 필터(너무 짧거나 너무 긴 쌍 제거)
df = df[(df['Q'].str.len().between(2, 100)) & (df['A'].str.len().between(2, 120))]
len(df), df.head(3)


(11740,
               Q            A
 0        12시 땡!   하루가 또 가네요.
 1   1지망 학교 떨어졌어    위로해 드립니다.
 2  3박4일 놀러가고 싶다  여행은 언제나 좋죠.)

In [7]:
# 학습/ 검증 분리

from sklearn.model_selection import train_test_split
train_df, valid_df = train_test_split(df, test_size=0.05, random_state=42, shuffle=True)
len(train_df), len(valid_df)

(11153, 587)

In [8]:
# 3) 토크나이저/모델 (KoGPT2)

from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "skt/kogpt2-base-v2"
tok = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

SEP = "<unused0>"   # KoGPT2에 남는 unused 토큰 하나를 구분자로 사용
bos, eos, pad = tok.bos_token, tok.eos_token, tok.pad_token or "<pad>"

if tok.pad_token is None:
    tok.add_special_tokens({"pad_token": "<pad>"})
    model.resize_token_embeddings(len(tok))


config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [9]:
#4) 데이터셋/콜레이터

import torch
from torch.utils.data import Dataset

class ChatDataset(Dataset):
    def __init__(self, frame, tokenizer, max_len=128):
        self.q = frame['Q'].tolist()
        self.a = frame['A'].tolist()
        self.tok = tokenizer
        self.max_len = max_len
    def __len__(self): return len(self.q)
    def __getitem__(self, i):
        q, a = self.q[i], self.a[i]
        text = f"{bos}{q}{SEP}{a}{eos}"
        enc = self.tok(text, truncation=True, max_length=self.max_len, return_tensors=None)
        # 레이블 마스킹: SEP 이전까지 -100
        ids = enc['input_ids']
        sep_id = self.tok.convert_tokens_to_ids(SEP)
        try:
            sep_pos = ids.index(sep_id)
        except ValueError:
            sep_pos = len(ids)//2  # 혹시 못 찾으면 대충 절반(명시적 가정)
        labels = [-100]* (sep_pos+1) + ids[sep_pos+1:]
        labels = labels[:self.max_len]
        ids = ids[:self.max_len]
        attn = [1]*len(ids)
        # 패딩
        pad_id = self.tok.pad_token_id
        while len(ids) < self.max_len:
            ids.append(pad_id); attn.append(0); labels.append(-100)
        return {"input_ids": torch.tensor(ids), "attention_mask": torch.tensor(attn), "labels": torch.tensor(labels)}

train_ds = ChatDataset(train_df, tok)
valid_ds = ChatDataset(valid_df, tok)
len(train_ds), len(valid_ds)


(11153, 587)

In [10]:
# 학습
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="kogpt2-chatbot",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    fp16=torch.cuda.is_available(),
    num_train_epochs=3,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_steps=100,
    save_steps=1000,
    save_total_limit=2,
    report_to="none"
)

trainer = Trainer(model=model, args=args, train_dataset=train_ds, eval_dataset=valid_ds)
trainer.train()



/opt/conda/lib/python3.12/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
500,2.633000,2.530363
1000,1.780300,2.223514
1500,1.117900,2.032925
2000,1.059300,1.985633


TrainOutput(global_step=2091, training_loss=1.8864070978078016, metrics={'train_runtime': 577.7543, 'train_samples_per_second': 57.912, 'train_steps_per_second': 3.619, 'total_flos': 2184532033536000.0, 'train_loss': 1.8864070978078016, 'epoch': 2.9978494623655916})

In [11]:
# 간단 인퍼런스 함수

@torch.no_grad()
def chat(prompt, max_new_tokens=64, top_p=0.9, temperature=0.8):
    model.eval()
    x = f"{bos}{prompt}{SEP}"
    inp = tok(x, return_tensors="pt").to(model.device)
    out = model.generate(
        **inp,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_p=top_p,
        temperature=temperature,
        pad_token_id=tok.pad_token_id,
        eos_token_id=tok.eos_token_id
    )
    text = tok.decode(out[0], skip_special_tokens=False)
    # SEP 이후부터 eos 전까지 잘라내기
    try:
        ans = text.split(SEP,1)[1]
    except:
        ans = text
    ans = ans.replace(tok.eos_token or "</s>", "").replace(tok.bos_token or "<s>", "").strip()
    return ans

print(chat("오늘 기분이 좀 우울한데 뭐 하면 좋을까?"))


마음의 안정을 취하기 좋은 날이죠.


In [12]:
# 7) 저장 & 로드

trainer.save_model("kogpt2-chatbot/best")
tok.save_pretrained("kogpt2-chatbot/best")
# 로드
# model = AutoModelForCausalLM.from_pretrained("kogpt2-chatbot/best"); tok = AutoTokenizer.from_pretrained("kogpt2-chatbot/best")


('kogpt2-chatbot/best/tokenizer_config.json',
 'kogpt2-chatbot/best/special_tokens_map.json',
 'kogpt2-chatbot/best/vocab.json',
 'kogpt2-chatbot/best/merges.txt',
 'kogpt2-chatbot/best/added_tokens.json',
 'kogpt2-chatbot/best/tokenizer.json')

In [13]:
print(chat("오늘 저녁에 뭘 먹을까?"))

맛있는 거 드세요.


In [14]:
print(chat("너는 누구야?"))

저는 마음을 이어주는 위로봇입니다.


In [15]:
print(chat("다른 이름은 없어?"))

저는 위로봇입니다.


In [16]:
print(chat("ai는 어떻게 공부하는게 좋아?"))

먼저 관심사를 알아보고 기술을 알아보는게 우선이에요.


In [17]:
print(chat("어려워"))

많이 힘들어요.


In [18]:
print(chat("네 대답의 정확도는?"))

질질 끌지 마세요.


In [19]:
print(chat("어떤 질문이 가능해?"))

저는 답이 없어요.


In [22]:
print(chat("Q의 내용이 총 몇가지가 있어?"))

그 중에 마음에 드는 건 사줄 거예요.
